In [1]:
# Import the random and numpy module 
import pandas as pd
import random
import numpy as np
import timeit
from citipy import citipy
import requests
# Import the API key.
from config import weather_api_key

# Import the datetime module from the datetime library.
from datetime import datetime

In [2]:
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs
coordinates =list(lat_lngs)

In [3]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

749

In [4]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [5]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1


Beginning Data Retrieval     
-----------------------------


In [6]:
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        # city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Description": city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
    

Processing Record 1 of Set 1 | punta arenas
Processing Record 2 of Set 1 | margate
Processing Record 3 of Set 1 | haines junction
Processing Record 4 of Set 1 | erenhot
Processing Record 5 of Set 1 | coaldale
Processing Record 6 of Set 1 | bredasdorp
Processing Record 7 of Set 1 | kalomo
City not found. Skipping...
Processing Record 8 of Set 1 | tuatapere
Processing Record 9 of Set 1 | alyangula
Processing Record 10 of Set 1 | mezhdurechenskiy
Processing Record 11 of Set 1 | tiksi
Processing Record 12 of Set 1 | busselton
Processing Record 13 of Set 1 | albany
Processing Record 14 of Set 1 | barrow
Processing Record 15 of Set 1 | saskylakh
Processing Record 16 of Set 1 | nikolskoye
Processing Record 17 of Set 1 | hami
Processing Record 18 of Set 1 | rikitea
Processing Record 19 of Set 1 | esperance
Processing Record 20 of Set 1 | zhigansk
Processing Record 21 of Set 1 | cabo san lucas
Processing Record 22 of Set 1 | kishi
Processing Record 23 of Set 1 | mareeba
Processing Record 24 of 

In [10]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,Punta Arenas,CL,-53.1500,-70.9167,41.11,65,75,16.11,broken clouds
1,Margate,GB,51.3813,1.3862,62.31,80,100,4.00,overcast clouds
2,Haines Junction,CA,60.7522,-137.5108,46.35,57,99,4.61,overcast clouds
3,Erenhot,CN,43.6475,111.9767,48.72,29,49,11.23,scattered clouds
4,Coaldale,CA,49.7167,-112.6185,67.32,41,20,14.97,few clouds
...,...,...,...,...,...,...,...,...,...
683,Dusti,TJ,37.3481,68.6640,66.92,31,97,1.21,overcast clouds
684,Along,IN,28.1667,94.7667,74.34,97,64,0.51,broken clouds
685,Jacareacanga,BR,-6.2222,-57.7528,73.69,96,100,0.72,overcast clouds
686,Castres,FR,43.5833,2.1667,55.17,87,0,4.61,clear sky


In [11]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")